In [1]:
!nvcc --version
!pip3 install pycuda

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660393 sha256=cd8ba5de6d978049aa51d2eded645b5a8c876e39832588a48c11cb9dd2897649
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [2]:
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit
import numpy as np

In [4]:
kernel_code = """
#include <curand_kernel.h>

extern "C" __global__ void generate_random_numbers(float* numbers, int n, int seed) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;

  if (idx < n) {
    curandState state;
    curand_init(seed, idx, 0, &state);
    numbers[idx] = curand_uniform(&state);
  }
}

extern "C" __global__ void calc_positional_encoding(float* pos_enc, int num_rows, int num_cols, int embedding_dimension) {
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < num_rows && col < num_cols
  pos_enc = (current_dim & 1) ? sin(token_idx_in_sentence / (10000 * pow((2 * current_dim) / embedding_dimension) :
                                cos(token_idx_in_sentence / (10000 * pow((2 * current_dim) / embedding_dimension)));
}
"""

mod = SourceModule(kernel_code,
                   no_extern_c=True,  # This is important!
                   options=["-std=c++11",
                           "-Xcompiler",
                           "-fPIC"])

In [5]:
sentence = "This is a sentence"
vocab = ["This", "is", "a", "sentence"]
sentence_toks = [0, 1, 2, 3] # Straight forward
word2tok = {"This" : 0, "is" : 1, "a" : 2, "sentence" : 3}

batch_size =
# Create the embedding matrix
vocab_size = len(vocab)
embedding_dimension = 2 # num of dimensions in each vector in the embedding matrix
embedding_num_elements = vocab_size * embedding_dimension
embedding_size_bytes = embedding_num_elements * np.float32().nbytes
embedding_matrix_gpu = cuda.mem_alloc(embedding_size_bytes)

generate_random_numbers = mod.get_function("generate_random_numbers")
generate_random_numbers(embedding_matrix_gpu, np.int32(embedding_num_elements), np.int32(0), block=(256, 1, 1), grid=(int(np.ceil(embedding_num_elements / 256)), 1))

embedding_matrix_host = np.empty(embedding_num_elements, dtype=np.float32)
cuda.memcpy_dtoh(embedding_matrix_host, embedding_matrix_gpu)

# print(embedding_matrix_host)

# We need to build a method to lookup tokens in the embedding matrix
# Use word2tok

# Create Positional Encoding
# embedding_dimension
pos_encodings_batch_size = 512
pos_encodings_num_elements = pos_encodings_batch_size * embedding_dimension
pos_encodings_size_bytes = pos_encodings_num_elements * np.float32().nbytes
pos_encodings_gpu = cuda.mem_alloc(pos_encodings_size_bytes)

calc_positional_encoding = mod.get_function("calc_positional_encoding")
